In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.1.0


In [2]:
source_data = pd.read_csv('./气候数据集/DailyDelhiClimateTrain.csv')

In [14]:
# 生成1-1000的tf数组（可迭代对象）
range_ds = tf.data.Dataset.range(1002)
# 通过batch形式获取数据,drop_remainder 是剪掉尾巴
batchs = range_ds.batch(5,drop_remainder=True)
# take(4)代表只取五组数据
for i in batchs.take(5):
    print(i.numpy())

[0 1 2 3 4]
[5 6 7 8 9]
[10 11 12 13 14]
[15 16 17 18 19]
[20 21 22 23 24]


### 当需要一步一步获取数据和标签的时候

In [16]:
def dense_1_step(batch):
  # Shift features and labels one step relative to each other.
  # 切分batch数组以指定的形式返回
    return batch[:-1], batch[-1]

predict_dense_1_step = batchs.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
    print(features.numpy(), " => ", label.numpy())

[0 1 2 3]  =>  4
[5 6 7 8]  =>  9
[10 11 12 13]  =>  14


### 当需要用类似windows移动的方式获取数据的时候

In [17]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
    """
    ds: 数据集
    window_size: 窗口的尺寸
    shift: 窗口的移动单位
    stride: 取的数据的间隔
    """
    windows = ds.window(window_size+1, shift=shift, stride=stride)

    def sub_to_batch(sub):
        return sub.batch(window_size+1, drop_remainder=True)

    windows = windows.flat_map(sub_to_batch)
    return windows


In [19]:
windows_dataset = make_window_dataset(range_ds)
# 获取窗口的batch数据集,然后使用lambda函数获取标签
process_windows_dataset = windows_dataset.map(dense_1_step)
for features, label in process_windows_dataset.take(5):
    print(features.numpy(), " => ", label.numpy())

[0 1 2 3]  =>  4
[1 2 3 4]  =>  5
[2 3 4 5]  =>  6
[3 4 5 6]  =>  7
[4 5 6 7]  =>  8


### 当需要对数据进行随机采样

In [21]:
windows_dataset = make_window_dataset(range_ds)
# 对batch数据集进行shuffle操作
windows_dataset = windows_dataset.shuffle(200)
# 获取标签
windows_dataset = windows_dataset.map(dense_1_step)

for features, label in windows_dataset.take(5):
    print(features.numpy(), " => ", label.numpy())

[ 99 100 101 102]  =>  103
[148 149 150 151]  =>  152
[126 127 128 129]  =>  130
[155 156 157 158]  =>  159
[ 8  9 10 11]  =>  12


### 当需要输入模型时

In [23]:
windows_dataset = make_window_dataset(range_ds)
# 对batch数据集进行shuffle操作
windows_dataset = windows_dataset.shuffle(200)
# 获取标签
windows_dataset = windows_dataset.map(dense_1_step)
# 再次batch，这次batch是将已经分好标签的数据进行打包
"""
tf.data API 通过 tf.data.Dataset.prefetch 转换提供了一种软件流水线机制，该机制可用于将生成数据的时间和使用数据的时间分离开。
具体而言，该转换使用后台线程和内部缓冲区，以便在请求元素之前从输入数据集中预取这些元素
因此，为了实现上图所示的流水线效果，您可以将 prefetch() 作为最终转换添加到数据集流水线中（如果单步训练使用 n 个元素，则添加 prefetch(n)）
"""
windows_dataset = windows_dataset.batch(6).prefetch(1)
for features, label in windows_dataset.take(5):
    print(features.numpy(), " => ", label.numpy())

[[144 145 146 147]
 [ 77  78  79  80]
 [ 47  48  49  50]
 [ 91  92  93  94]
 [140 141 142 143]
 [ 50  51  52  53]]  =>  [148  81  51  95 144  54]
[[176 177 178 179]
 [ 16  17  18  19]
 [201 202 203 204]
 [108 109 110 111]
 [ 73  74  75  76]
 [ 71  72  73  74]]  =>  [180  20 205 112  77  75]
[[198 199 200 201]
 [107 108 109 110]
 [  7   8   9  10]
 [ 68  69  70  71]
 [ 85  86  87  88]
 [ 99 100 101 102]]  =>  [202 111  11  72  89 103]
[[165 166 167 168]
 [ 10  11  12  13]
 [ 43  44  45  46]
 [119 120 121 122]
 [138 139 140 141]
 [ 66  67  68  69]]  =>  [169  14  47 123 142  70]
[[ 56  57  58  59]
 [167 168 169 170]
 [185 186 187 188]
 [103 104 105 106]
 [126 127 128 129]
 [ 60  61  62  63]]  =>  [ 60 171 189 107 130  64]
